![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Setting Up Your Environment

## Learning Objectives (20 minutes)
By the end of this notebook, you will be able to:
1. **Install and configure** all required services locally
2. **Set up environment variables** correctly with consistent defaults
3. **Verify service connectivity** and health status
4. **Troubleshoot** common setup issues
5. **Prepare your environment** for the remaining course sections

## Prerequisites
- Docker and Docker Compose installed
- Python 3.10+ environment
- OpenAI API key obtained
- Completed previous notebooks in Section 1

---

## Overview

This notebook will guide you through setting up the complete development environment for the Context Engineering course. We'll configure:

- **Redis 8**: Vector database and state storage
- **Agent Memory Server**: Long-term memory management
- **Python Environment**: Course dependencies and packages
- **Environment Variables**: Consistent configuration
- **Health Checks**: Verify everything is working

## System Requirements Check

Let's start by checking that your system meets the requirements:

In [ ]:
import sys
import subprocess
import os
from pathlib import Path

def check_requirement(name, command, min_version=None):
    """Check if a system requirement is met."""
    try:
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            version = result.stdout.strip()
            print(f"✅ {name}: {version}")
            return True
        else:
            print(f"❌ {name}: Not found")
            return False
    except Exception as e:
        print(f"❌ {name}: Error checking - {e}")
        return False

print("🔍 System Requirements Check")
print("=" * 40)

# Check Python version
python_version = f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}"
if sys.version_info >= (3, 10):
    print(f"✅ Python: {python_version}")
else:
    print(f"⚠️  Python: {python_version} (3.10+ recommended)")

# Check other requirements
requirements = [
    ("Docker", "docker --version"),
    ("Docker Compose", "docker-compose --version"),
    ("Git", "git --version")
]

all_good = True
for name, command in requirements:
    if not check_requirement(name, command):
        all_good = False

print("\n" + "=" * 40)
if all_good:
    print("🎉 All system requirements met!")
else:
    print("⚠️  Some requirements missing. Please install before continuing.")

## Environment Configuration

Let's set up the environment variables with consistent defaults:

In [ ]:
import os
from dotenv import load_dotenv
import getpass

# Load existing environment variables
load_dotenv()

# Define consistent defaults (matching docker-compose.yml)
ENV_DEFAULTS = {
    "REDIS_URL": "redis://localhost:6379",
    "AGENT_MEMORY_URL": "http://localhost:8088",  # External port from docker-compose
    "OPENAI_API_KEY": None  # Must be provided by user
}

print("🔧 Environment Configuration")
print("=" * 40)

# Set up each environment variable
for key, default in ENV_DEFAULTS.items():
    current_value = os.getenv(key)
    
    if current_value:
        print(f"✅ {key}: Already set")
        continue
    
    if default:
        os.environ[key] = default
        print(f"🔧 {key}: Set to default ({default})")
    else:
        # Special handling for API key
        if key == "OPENAI_API_KEY":
            try:
                # Try to get from user input (works in interactive environments)
                api_key = getpass.getpass(f"Please enter your {key}: ")
                if api_key.strip():
                    os.environ[key] = api_key.strip()
                    print(f"✅ {key}: Set successfully")
                else:
                    print(f"⚠️  {key}: Not provided (some features will be limited)")
            except (EOFError, KeyboardInterrupt):
                print(f"⚠️  {key}: Not provided (some features will be limited)")

print("\n📋 Current Environment:")
for key in ENV_DEFAULTS.keys():
    value = os.getenv(key)
    if key == "OPENAI_API_KEY" and value:
        # Mask the API key for security
        masked_value = f"{value[:8]}...{value[-4:]}" if len(value) > 12 else "***"
        print(f"  {key}: {masked_value}")
    else:
        print(f"  {key}: {value or 'Not set'}")

## Service Setup with Docker Compose

Now let's start the required services using Docker Compose:

In [ ]:
import subprocess
import time
import requests

def run_command(command, description):
    """Run a shell command and return success status."""
    print(f"🔄 {description}...")
    try:
        result = subprocess.run(command, shell=True, capture_output=True, text=True, cwd="../..")
        if result.returncode == 0:
            print(f"✅ {description} completed")
            return True
        else:
            print(f"❌ {description} failed:")
            print(f"   Error: {result.stderr}")
            return False
    except Exception as e:
        print(f"❌ {description} failed: {e}")
        return False

print("🐳 Starting Services with Docker Compose")
print("=" * 50)

# Check if docker-compose.yml exists
compose_file = Path("../../docker-compose.yml")
if not compose_file.exists():
    print(f"❌ docker-compose.yml not found at {compose_file.absolute()}")
    print("   Please ensure you're running from the correct directory.")
else:
    print(f"✅ Found docker-compose.yml at {compose_file.absolute()}")
    
    # Start services
    if run_command("docker-compose up -d", "Starting services"):
        print("\n⏳ Waiting for services to start...")
        time.sleep(10)  # Give services time to start
        
        # Check service status
        run_command("docker-compose ps", "Checking service status")
    else:
        print("\n💡 Troubleshooting tips:")
        print("   1. Make sure Docker is running")
        print("   2. Check if ports 6379 and 8088 are available")
        print("   3. Try: docker-compose down && docker-compose up -d")

## Health Checks

Let's verify that all services are running correctly:

In [ ]:
import redis
import requests
import json

def check_redis_health():
    """Check Redis connectivity."""
    try:
        r = redis.from_url(os.getenv("REDIS_URL"))
        r.ping()
        info = r.info()
        version = info.get('redis_version', 'unknown')
        print(f"✅ Redis: Connected (version {version})")
        return True
    except Exception as e:
        print(f"❌ Redis: Connection failed - {e}")
        return False

def check_agent_memory_server():
    """Check Agent Memory Server health."""
    try:
        url = f"{os.getenv('AGENT_MEMORY_URL')}/v1/health"
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            health_data = response.json()
            print(f"✅ Agent Memory Server: Healthy")
            print(f"   Status: {health_data.get('status', 'unknown')}")
            return True
        else:
            print(f"❌ Agent Memory Server: HTTP {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"❌ Agent Memory Server: Connection failed - {e}")
        return False

def check_openai_key():
    """Check OpenAI API key validity."""
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("⚠️  OpenAI API Key: Not set (some features will be limited)")
        return False
    
    if api_key.startswith("sk-") and len(api_key) > 20:
        print("✅ OpenAI API Key: Format looks correct")
        return True
    else:
        print("⚠️  OpenAI API Key: Format may be incorrect")
        return False

print("🏥 Health Checks")
print("=" * 30)

# Run all health checks
checks = [
    ("Redis", check_redis_health),
    ("Agent Memory Server", check_agent_memory_server),
    ("OpenAI API Key", check_openai_key)
]

results = []
for name, check_func in checks:
    try:
        result = check_func()
        results.append(result)
    except Exception as e:
        print(f"❌ {name}: Unexpected error - {e}")
        results.append(False)

print("\n" + "=" * 30)
passed = sum(results)
total = len(results)

if passed == total:
    print(f"🎉 All health checks passed! ({passed}/{total})")
    print("   Your environment is ready for the course.")
elif passed >= 2:  # Redis + AMS are critical
    print(f"✅ Core services ready ({passed}/{total})")
    print("   You can proceed with most course content.")
else:
    print(f"⚠️  Some services need attention ({passed}/{total})")
    print("   Please check the troubleshooting section below.")

## Install Course Dependencies

Let's install the Redis Context Course package and verify it works:

In [ ]:
# Install the Redis Context Course package in development mode
print("📦 Installing Course Dependencies")
print("=" * 40)

# Install the reference agent package
%pip install --upgrade -q -e ../../reference-agent

print("✅ Package installation completed")

# Test imports
try:
    from redis_context_course.models import Course, StudentProfile, DifficultyLevel, CourseFormat
    from redis_context_course.course_manager import CourseManager
    from redis_context_course.redis_config import redis_config
    print("✅ Core modules imported successfully")
    
    # Test Redis connection through the package
    if redis_config.health_check():
        print("✅ Package Redis connection working")
    else:
        print("⚠️  Package Redis connection failed")
        
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("   Please check the package installation.")
except Exception as e:
    print(f"⚠️  Connection test failed: {e}")

## Generate Sample Data

Let's create and ingest sample course data for the exercises:

In [ ]:
import subprocess
import os

print("📚 Generating Sample Course Data")
print("=" * 40)

# Change to reference agent directory
ref_agent_dir = "../../reference-agent"

try:
    # Generate course data
    print("🔄 Generating course catalog...")
    result = subprocess.run(
        ["python", "-m", "redis_context_course.scripts.generate_courses"],
        cwd=ref_agent_dir,
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        print("✅ Course catalog generated")
        
        # Ingest course data
        print("🔄 Ingesting courses into Redis...")
        result = subprocess.run(
            ["python", "-m", "redis_context_course.scripts.ingest_courses"],
            cwd=ref_agent_dir,
            capture_output=True,
            text=True
        )
        
        if result.returncode == 0:
            print("✅ Courses ingested successfully")
            
            # Verify data was ingested
            try:
                course_manager = CourseManager()
                # Try a simple search to verify data
                results = await course_manager.search_courses("programming", limit=1)
                if results:
                    print(f"✅ Data verification: Found {len(results)} course(s)")
                else:
                    print("⚠️  Data verification: No courses found")
            except Exception as e:
                print(f"⚠️  Data verification failed: {e}")
        else:
            print(f"❌ Course ingestion failed: {result.stderr}")
    else:
        print(f"❌ Course generation failed: {result.stderr}")
        
except Exception as e:
    print(f"❌ Data setup failed: {e}")
    print("\n💡 You can manually run these commands later:")
    print(f"   cd {ref_agent_dir}")
    print("   python -m redis_context_course.scripts.generate_courses")
    print("   python -m redis_context_course.scripts.ingest_courses")

## ✅ Knowledge Check: Environment Setup

**Question 1**: Which port does the Agent Memory Server use for external access?
- [ ] 6379
- [ ] 8000
- [ ] 8088
- [ ] 3000

**Question 2**: What health check endpoint should you use for the Agent Memory Server?
- [ ] /health
- [ ] /v1/health
- [ ] /status
- [ ] /ping

**Question 3**: Which command generates sample course data?
- [ ] python -m redis_context_course.scripts.setup_data
- [ ] python -m redis_context_course.scripts.generate_courses
- [ ] python -m redis_context_course.scripts.create_catalog
- [ ] python -m redis_context_course.scripts.init_data

*Answers: 1-C, 2-B, 3-B*

## Test Your Setup

Let's run a quick test to make sure everything is working:

In [ ]:
print("🧪 Testing Your Setup")
print("=" * 30)

# Test 1: Course search
try:
    course_manager = CourseManager()
    results = await course_manager.search_courses("computer science", limit=3)
    print(f"✅ Course search: Found {len(results)} courses")
    for course in results[:2]:  # Show first 2
        print(f"   • {course.course_code}: {course.title}")
except Exception as e:
    print(f"❌ Course search failed: {e}")

# Test 2: Memory client (if available)
try:
    from agent_memory_client import MemoryAPIClient, MemoryClientConfig
    
    config = MemoryClientConfig(
        base_url=os.getenv("AGENT_MEMORY_URL"),
        default_namespace="redis_university_test"
    )
    memory_client = MemoryAPIClient(config=config)
    
    # Simple health check
    # Note: This might fail if AMS is not running, which is OK for now
    print("✅ Memory client: Initialized successfully")
    
except ImportError:
    print("⚠️  Memory client: Not available (will be covered in Section 3)")
except Exception as e:
    print(f"⚠️  Memory client: {e}")

# Test 3: Student profile creation
try:
    student = StudentProfile(
        name="Test Student",
        email="test@university.edu",
        major="Computer Science",
        year=2,
        completed_courses=["CS101"],
        interests=["machine learning"],
        preferred_format=CourseFormat.ONLINE,
        preferred_difficulty=DifficultyLevel.INTERMEDIATE
    )
    print(f"✅ Student profile: Created for {student.name}")
except Exception as e:
    print(f"❌ Student profile failed: {e}")

print("\n🎉 Setup testing completed!")

## Troubleshooting Guide

If you encounter issues, here are common solutions:

### Redis Connection Issues
**Problem**: `ConnectionError: Error connecting to Redis`

**Solutions**:
1. Check if Redis is running: `docker ps | grep redis`
2. Restart Redis: `docker-compose restart redis`
3. Check port availability: `netstat -an | grep 6379`
4. Verify REDIS_URL: Should be `redis://localhost:6379`

### Agent Memory Server Issues
**Problem**: `Connection refused` on port 8088

**Solutions**:
1. Check if AMS is running: `docker ps | grep agent-memory-server`
2. Restart AMS: `docker-compose restart agent-memory-server`
3. Check logs: `docker-compose logs agent-memory-server`
4. Verify URL: Should be `http://localhost:8088`

### OpenAI API Issues
**Problem**: `Invalid API key` or `Rate limit exceeded`

**Solutions**:
1. Verify your API key at https://platform.openai.com/api-keys
2. Check your usage limits and billing
3. Ensure key starts with `sk-` and is properly set

### Package Import Issues
**Problem**: `ModuleNotFoundError: No module named 'redis_context_course'`

**Solutions**:
1. Reinstall package: `pip install -e ../../reference-agent`
2. Check Python path: `sys.path`
3. Restart Jupyter kernel

### Docker Issues
**Problem**: `docker-compose` command not found

**Solutions**:
1. Try `docker compose` (newer syntax)
2. Install Docker Compose: https://docs.docker.com/compose/install/
3. Check Docker is running: `docker version`

## 🛠️ Hands-on Exercise: Environment Verification

**Task**: Complete these verification steps to ensure your environment is ready:

1. **Service Status**: Run `docker-compose ps` and verify all services are "Up"
2. **Redis Test**: Connect to Redis and run a simple command
3. **Course Search**: Search for "programming" courses and get results
4. **Memory Test**: Try creating a simple memory record (if AMS is running)

**Expected Time**: 10 minutes  
**Deliverable**: Verification checklist completion

### Your Verification Results:
*(Check off completed items)*

- [ ] All Docker services running
- [ ] Redis connection successful
- [ ] Course search returns results
- [ ] Memory client initializes (if AMS available)
- [ ] No import errors for course packages

## Summary

Congratulations! You've successfully set up your development environment for the Context Engineering course. Here's what we accomplished:

### ✅ Completed Setup
- **System Requirements**: Verified Python, Docker, and other dependencies
- **Environment Variables**: Configured consistent defaults for all services
- **Services**: Started Redis and Agent Memory Server with Docker Compose
- **Health Checks**: Verified all services are running correctly
- **Course Package**: Installed and tested the Redis Context Course package
- **Sample Data**: Generated and ingested course catalog for exercises

### 🔧 Key Configuration
- **Redis URL**: `redis://localhost:6379`
- **Agent Memory URL**: `http://localhost:8088`
- **Health Endpoint**: `/v1/health`
- **Package**: `redis-context-course` installed in development mode

### 🚀 Ready for Next Steps
Your environment is now ready for the remaining course sections:

- **Section 2**: System Context - Learn to craft system prompts and define tools
- **Section 3**: Memory Management - Explore working and long-term memory
- **Section 4**: Optimizations - Master advanced context engineering techniques

## Need Help?

If you encounter any issues:
1. **Check the troubleshooting guide** above
2. **Review the health check results** for specific error messages
3. **Consult the course documentation** in the reference agent README
4. **Ask for help** in the course discussion forum

---

**🎉 Environment setup complete! You're ready to dive into context engineering!**